In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import time
import plotly.io as pio
import plotly.graph_objs as go
import plotly.express as px
pio.renderers.default = 'notebook'

class StockInfo():
    def __init__(self,filename,t='y'):
        if t not in ['y','q']:
            print('Select "q" for quarterly || "y" for yearly.')
            self.state=0
            return
        if t == 'q':
            self.t = 'quarterly'
        else:
            self.t = 'yearly'
        try:
            with open(filename,'r') as file:
                self.stocks = [symbol.strip() for symbol in file.readlines()]
                if len(self.stocks) > 0:
                    self.get_data()
                    self.state = 1
                else:
                    self.state=0
        except Exception as e:
            print(f'✖ {filename} not found: {e}.')
            self.state = 0
            return
    
    def get_data(self):
        delete = []
        for stock in self.stocks:
            try:
                pd.read_csv(f'{stock}_financials-{self.t}.csv')
            except FileNotFoundError:
                try:
                    ticker = yf.Ticker(stock)
                    cashflow_statement = ticker.get_cashflow(freq=self.t).transpose()
                    freecashflow_df = cashflow_statement['FreeCashFlow'].dropna(axis=0)
                    income_statement = ticker.get_incomestmt(freq=self.t).transpose()
                    financials_df = income_statement[['TotalRevenue','NetIncome']].dropna(axis=0)
                    if self.t == 'quarterly':
                        earnings_df = ticker.get_earnings_history()[['epsActual','epsEstimate']].iloc[::-1]
                        merged_df = pd.concat([freecashflow_df,financials_df,earnings_df],join='inner',axis=1).iloc[::-1]
                    else:
                        merged_df = pd.concat([freecashflow_df,financials_df],join='inner',axis=1).iloc[::-1]
                    merged_df.to_csv(f'{stock}_financials-{self.t}.csv')
                    time.sleep(0.2)
                except Exception as e:
                    delete.append(stock)
                    print(f'✖ Could not fetch financials: {e}.')
                    continue
        for stock in delete:
            self.stocks.remove(stock)
                    
    def get_industry(self):
        self.sectors = pd.Series(dtype='object')
        for stock in self.stocks:
            try:
                ticker = yf.Ticker(stock)
                self.sectors.loc[stock] = ticker.get_info()['industry']
                time.sleep(0.2)
            except Exception as e:
                print(f'✖ Could not fetch the industry for {stock}: {e}.')
                continue
                
    def plot_earnings(self):
        if self.state == 0:
            return
        if self.t == 'yearly':
            print('\n✖ Earnings only quarterly available.')
            return
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                df.index = df.index.to_period('Q').astype('str')
            else:
                df.index = df.index.year.astype('str')
            for row,data in df.iterrows():
                if data['epsActual'] > data['epsEstimate']:
                    df.loc[row,'color'] = 'lime'
                elif data['epsActual'] == data['epsEstimate']:
                    df.loc[row,'color'] = 'royalblue'
                else:
                    df.loc[row,'color'] = 'red'
            
            fig = go.Figure()
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df['epsEstimate'],
                    name='Estimate',
                    mode='markers',
                    marker=dict(size=20,color='#000',line=dict(color='#999',width=3))
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df['epsActual'],
                    name='Actual',
                    mode='markers',
                    marker=dict(size=18,color=df['color'])
                )
            )
            fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#000',
                xaxis=dict(gridcolor='#222'),
                yaxis=dict(gridcolor='#222'),
                font=dict(color='#fff'),
                title=f'{stock} quarterly EPS',
                showlegend=False
            )
            fig.show()
    
    def plot_growth(self):
        if self.state == 0:
            return
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                df.index = df.index.to_period('Q').astype('str')
            else:
                df.index = df.index.year.astype('str')
                
            trend = pd.DataFrame(columns=['FCF-Margin','Net-Margin'])
            trend['FCF-Margin'] = df['FreeCashFlow']/df['TotalRevenue']*100
            trend['Net-Margin'] = df['NetIncome']/df['TotalRevenue']*100
             
            fig = go.Figure()
            fig.add_trace(
                go.Bar(
                    x=trend.index,
                    y=trend['FCF-Margin'],
                    marker=dict(color='mediumpurple',line=dict(color='rebeccapurple',width=2)),
                    name='FCF-Margin'
                )
            )
            fig.add_trace(
                go.Bar(
                    x=trend.index,
                    y=trend['Net-Margin'],
                    marker=dict(color='deepskyblue',line=dict(color='steelblue',width=2)),
                    name='Net-Margin'
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=(df['TotalRevenue'].pct_change()*100).fillna(0.0),
                    mode='lines+markers',
                    line=dict(color='orange',width=3),
                    marker=dict(color='orange',size=10,line=dict(color='#000',width=1)),
                    name='ChangeInRevenue'
                )
            )
            fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#222',
                xaxis=dict(gridcolor='#333'),
                yaxis=dict(gridcolor='#333',title='%'),
                font=dict(color='#fff'),
                title=f'{stock} Relative Growth'
            )
            fig.show()
            
    def plot_diversification(self):
        if self.state == 0:
            return
        self.get_industry()
        fig = go.Figure()
        values = []
        
        for sector, stocks in self.sectors.groupby(self.sectors.values):
            values.append({
                'industry':sector,
                'data':{'nstocks':len(stocks),'stocks':[stock.strip() for stock in stocks.index]}
            })
        cm = px.colors.sequential.Viridis
        fig.add_trace(
            go.Pie(
                labels=[i['industry'] for i in values],
                values=[i['data']['nstocks'] for i in values],
                name='Stocks',
                customdata=[i['data']['stocks'] for i in values],
                hovertemplate='%{customdata}',
                textinfo='none',
                marker=dict(colors=cm,line=dict(color='#000',width=1)),
                hole=0.6
            )
        )
        fig.update_layout(
                paper_bgcolor='#000',
                plot_bgcolor='#000',
                font=dict(color='#fff'),
                title='Diversification'
            )
        fig.show()
    
    def print_statistics(self):
        if self.state == 0:
            return
        stats_df = pd.DataFrame()
        
        for stock in self.stocks:
            df = pd.read_csv(f'{stock}_financials-{self.t}.csv',index_col=0,parse_dates=True)
            if self.t == 'quarterly':
                t = len(df)/4
            else:
                t = len(df)
            mean = (df['NetIncome'].pct_change()*100).dropna().mean()
            std = (df['NetIncome'].pct_change()*100).dropna().std()
            eff = (df['FreeCashFlow']/df['TotalRevenue']*100).mean()
            rfr = 4
            
            if df['NetIncome'].iloc[-1] <= 0 or df['NetIncome'].iloc[0] <= 0:
                cagr = np.NaN
                sr = np.NaN
            else:
                cagr = ((df['NetIncome'].iloc[-1]/df['NetIncome'].iloc[0])**(1/t)-1)*100
                sr = (mean-rfr)/std
            if mean <= 0:
                mean = np.NaN
                sr = np.NaN
            
            stats_df.loc[stock,'CAGR'] = round(cagr,2)
            stats_df.loc[stock,'AverageNetGrowth'] = round(mean,2)
            stats_df.loc[stock,'GrowthVolatility'] = round(std,2)
            stats_df.loc[stock,'CashEfficiency'] = round(eff,2)
            stats_df.loc[stock,'SharpRatio'] = round(sr,2)
            
        stats_df.sort_values(by='SharpRatio',ascending=False,inplace=True)
        print(f'\n{stats_df}')


if __name__ == '__main__':
    obj = StockInfo('WL.txt','y')
    obj.plot_earnings()
    obj.plot_growth()
    obj.plot_diversification()
    obj.print_statistics()

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import yfinance as yf
import time
pio.renderers.default = 'notebook'

class StockTechnical():
    def __init__(self,filepath,tf='1mo',period='5y'):
        self.tf = tf
        self.period = period
        try:
            with open (filepath,'r') as f:
                self.stocks = [i.strip() for i in f.readlines()]
                if len(self.stocks) > 0:
                    self.execute = True
                    self.get_data()
        except Exception as e:
            print(f'Error reading {filepath}: {e}.')
            self.execute = False
            return
        
    def get_data(self):
        if not self.execute:
            return
        allstocks = []
        for stock in self.stocks:
            try:
                df = pd.read_csv(f'{stock}_{self.tf}_{self.period}.csv',index_col=0,parse_dates=True)
                allstocks.append(df['Close'].rename(stock))
                print(f'{stock} data loaded.')
            except FileNotFoundError:
                try:
                    print(f'\nFeching data for {stock}...')
                    df = yf.download(tickers=stock,interval=self.tf,period=self.period,auto_adjust=True)
                    df.columns = ['Open','High','Low','Close','Volume']
                    df.index.name = 'Datetime'
                    df.to_csv(f'{stock}_{self.tf}_{self.period}.csv')
                    allstocks.append(df['Close'].rename(stock))
                    time.sleep(0.2)
                    print('✔ Data succefully downloaded.')
                except Exception as e:
                    print(f'✖ Could not fetch data for {stock}: {e}.')
                    continue
        self.stocks_df = pd.concat(allstocks,join='inner',axis=1)
        
    def print_statistics(self):
        if not self.execute:
            return
        returns = pd.DataFrame()
        for stock in self.stocks_df.columns:
            try:
                t = int(self.stocks_df.index.year[-1])-int(self.stocks_df.index.year[0])
                rfr = 4
                cagr = ((self.stocks_df[stock].iloc[-1]/self.stocks_df[stock].iloc[0])**(1/t)-1)*100
                if self.tf == '1d':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *252
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(252)
                    sr = mean/std
                elif self.tf == '1wk':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *52
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(52)
                    sr = (mean-rfr)/std
                elif self.tf == '1mo':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *12
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(12)
                    sr = (mean-rfr)/std
                else:
                    mean = np.NaN
                    std = np.NaN
                    sr = np.NaN

                returns.loc[stock,'CAGR'] = round(cagr,2)
                returns.loc[stock,'AverageReturn(y)'] = round(mean,2)
                returns.loc[stock,'ReturnVolatility(y)'] = round(std,2)
                returns.loc[stock,'SharpRatio'] = round(sr,2)

            except Exception as e:
                print(e)
        print(returns.sort_values(by='SharpRatio',ascending=False))
    
    def plot_returncorr(self):
        if self.execute == False:
            return
        fig = go.Figure()
        fig.add_trace(
            go.Heatmap(
                z=self.stocks_df.pct_change().dropna().corr(),
                y=self.stocks_df.columns,
                x=self.stocks_df.columns,
                colorscale='icefire',
                zmid=0
            )
        )
        fig.update_layout(
            paper_bgcolor='#000',
            plot_bgcolor='#000',
            font=dict(color='#fff',size=10),
            title=dict(text='Return Correlation Heatmap',font=dict(size=16,weight='bold'))
        )
        fig.show()
        
    def plot_performancebyyear(self):
        ovrrdf = pd.Series(dtype='float')
        scap = self.stocks_df.iloc[0].sum()
        ovrrdf.loc[self.stocks_df.index[0]] = 0.0
        for t, row in self.stocks_df.iloc[1:].iterrows():
            ovrrdf.loc[t] = round((row.sum()/scap-1)*100,2)
        ovrrdf.index = ovrrdf.index.astype('str')
        
        rolv = ovrrdf.rolling(14).std()
        
        fig = go.Figure()
        
        

In [ ]:
fig.add_trace(
            go.Scatter(
                x=ovrrdf.index,
                y=ovrrdf,
                mode='lines',
                line=dict(color='red',width=2),
                name='OverallReturn'
            )
        )

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import yfinance as yf
import time
pio.renderers.default = 'notebook'

class StockTechnical():
    def __init__(self,filepath,tf='1mo',period='5y'):
        self.tf = tf
        self.period = period
        try:
            with open (filepath,'r') as f:
                self.stocks = [i.strip() for i in f.readlines()]
                if len(self.stocks) > 0:
                    self.execute = True
                    self.get_data()
        except Exception as e:
            print(f'Error reading {filepath}: {e}.')
            self.execute = False
            return
        
    def get_data(self):
        if not self.execute:
            return
        allstocks = []
        for stock in self.stocks:
            try:
                df = pd.read_csv(f'{stock}_{self.tf}_{self.period}.csv',index_col=0,parse_dates=True)
                allstocks.append(df['Close'].rename(stock))
                print(f'{stock} data loaded.')
            except FileNotFoundError:
                try:
                    print(f'\nFeching data for {stock}...')
                    df = yf.download(tickers=stock,interval=self.tf,period=self.period,auto_adjust=True)
                    df.columns = ['Open','High','Low','Close','Volume']
                    df.index.name = 'Datetime'
                    df.to_csv(f'{stock}_{self.tf}_{self.period}.csv')
                    allstocks.append(df['Close'].rename(stock))
                    time.sleep(0.2)
                    print('✔ Data succefully downloaded.')
                except Exception as e:
                    print(f'✖ Could not fetch data for {stock}: {e}.')
                    continue
        self.stocks_df = pd.concat(allstocks,join='inner',axis=1)
        
    def print_statistics(self):
        if not self.execute:
            return
        returns = pd.DataFrame()
        for stock in self.stocks_df.columns:
            try:
                t = int(self.stocks_df.index.year[-1])-int(self.stocks_df.index.year[0])
                rfr = 4
                cagr = ((self.stocks_df[stock].iloc[-1]/self.stocks_df[stock].iloc[0])**(1/t)-1)*100
                if self.tf == '1d':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *252
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(252)
                    sr = mean/std
                elif self.tf == '1wk':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *52
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(52)
                    sr = (mean-rfr)/std
                elif self.tf == '1mo':
                    mean = (self.stocks_df[stock].pct_change()*100).mean() *12
                    std = (self.stocks_df[stock].pct_change()*100).std() *np.sqrt(12)
                    sr = (mean-rfr)/std
                else:
                    mean = np.NaN
                    std = np.NaN
                    sr = np.NaN

                returns.loc[stock,'CAGR'] = round(cagr,2)
                returns.loc[stock,'AverageReturn(y)'] = round(mean,2)
                returns.loc[stock,'ReturnVolatility(y)'] = round(std,2)
                returns.loc[stock,'SharpRatio'] = round(sr,2)

            except Exception as e:
                print(e)
        print(returns.sort_values(by='SharpRatio',ascending=False))
    
    def plot_returncorr(self):
        if self.execute == False:
            return
        fig = go.Figure()
        fig.add_trace(
            go.Heatmap(
                z=self.stocks_df.pct_change().dropna().corr(),
                y=self.stocks_df.columns,
                x=self.stocks_df.columns,
                colorscale='icefire',
                zmid=0
            )
        )
        fig.update_layout(
            paper_bgcolor='#000',
            plot_bgcolor='#000',
            font=dict(color='#fff',size=10),
            title=dict(text='Return Correlation Heatmap',font=dict(size=16,weight='bold'))
        )
        fig.show()
        
    def plot_performance(self):
        if self.tf == '1d':
            l= 85
        elif self.tf == '1wk':
            l= 12
        elif self.tf == '1mo':
            l = 3
        else:
            l = None
        ovrrdf = pd.Series(dtype='float')
        ovrrdf.loc[self.stocks_df.index[0]] = 0.0
        
        fig = go.Figure()
        
        scap = self.stocks_df.iloc[0].sum()
        for t, row in self.stocks_df.iloc[1:].iterrows():
            ovrrdf.loc[t] = round((row.sum()/scap-1)*100,2)
            
        ovrrdf.index = ovrrdf.index.astype('str')
        
        if l is not None:
            rolv = ovrrdf.rolling(l).std().dropna()
            ovrrdf = ovrrdf.iloc[l:]
            vbh = ovrrdf+rolv
            vbl = ovrrdf-rolv

            fig.add_trace(
                go.Scatter(
                    x=vbh.index,
                    y=vbh,
                    mode='lines',
                    line=dict(color='#666',width=1,dash='dot'),
                    name='UpsideRisk'
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=vbl.index,
                    y=vbl,
                    mode='lines',
                    line=dict(color='#666',width=1,dash='dot'),
                    name='DownsideRisk'
                )
            )
        fig.add_trace(
            go.Scatter(
                x=ovrrdf.index,
                y=ovrrdf,
                mode='lines',
                line=dict(color='red',width=2),
                name='OverallReturn'
            )
        )
        fig.update_layout(
            paper_bgcolor='#000',plot_bgcolor='#222',
            xaxis=dict(gridcolor='#333',title='Time'),
            yaxis=dict(gridcolor='#333',title='Return(%)'),
            font=dict(color='#fff',size=10),
            title=dict(text='Overall Return',font=dict(size=16,weight='bold'))
        )
        fig.show()
        
        
if __name__ == '__main__':
    obj = StockTechnical('WL.txt','1mo','5y')
    obj.plot_returncorr()
    obj.plot_performance()
    obj.print_statistics()